In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'country_demo', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())

print(emission_df.head())
results_temp = session.query(Temp_change)

#print(results) 

temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']

                       Entity  Code  Year  AnnualCO2emissions
0                      Africa  None  1961           161437399
1                        Asia  None  1961          1392745886
2  Asia (excl. China & India)  None  1961           721204193
3                       EU-27  None  1961          2157634122
4                       EU-28  None  1961          2746248923


In [7]:




season_df = temp_df.loc[temp_df["Months"].isin(selection)]

#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
# Calculate avg temp per Meteorological year
# meteor_df['avg_temp']= meteor_df.mean(axis =1)
# meteor_id_df = meteor_df.set_index('Area')

session.close() 

C:\Users\ubc\anaconda3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
# meteor_df.drop(columns=['field1','avg_temp'])
# meteor_df['avg_temp']= meteor_df.mean(axis =1)
# meteor_df[meteor_df["Area"] == "India"]

C:\Users\ubc\anaconda3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,field1,Area,Months,Element,Unit,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,avg_temp
1250,3534,India,Meteorological year,Temperature change,°C,-0.23,-0.531,-0.064,0.002,-0.06,...,0.235,0.364,0.307,0.419,0.576,0.977,0.866,0.769,0.685,59.113017


In [12]:
meteor_df_new = meteor_df.drop(columns=["field1", "Months", "Element", "Unit"])
meteor_df_new = meteor_df_new.set_index('Area')
meteor_df_new['avg_temp']= round(meteor_df_new.mean(axis =1),3)
meteor_df_new.loc["India"]

1961       -0.230
1962       -0.531
1963       -0.064
1964        0.002
1965       -0.060
1966        0.290
1967       -0.214
1968       -0.243
1969        0.300
1970        0.034
1971       -0.483
1972        0.014
1973        0.342
1974        0.014
1975       -0.317
1976       -0.023
1977        0.142
1978       -0.165
1979        0.281
1980        0.384
1981        0.040
1982       -0.067
1983       -0.206
1984       -0.167
1985        0.137
1986        0.120
1987        0.509
1988        0.436
1989       -0.083
1990       -0.146
1991        0.114
1992       -0.045
1993        0.134
1994        0.009
1995        0.133
1996        0.233
1997       -0.169
1998        0.493
1999        0.381
2000        0.274
2001        0.398
2002        0.735
2003        0.583
2004        0.475
2005        0.479
2006        0.578
2007        0.513
2008        0.167
2009        1.035
2010        1.017
2011        0.235
2012        0.364
2013        0.307
2014        0.419
2015        0.576
2016      

In [ ]:
month_df["Area"].tolist()



In [ ]:
country='United States of America'

#Get avg_temp change by season for the selected country
months_country_mean = month_df.loc[month_df['Area'] == country] 
months_country_mean.set_index("Months" , inplace=True)
months_country_mean

In [ ]:
#Drop unwanted fields
months_country_mean = months_country_mean.drop(['field1','Area', 'Element', 'Unit'], 1)
#Get years data
years = months_country_mean.columns
 

In [ ]:

months_country_mean

In [ ]:
months_list = []
# Get months for each country
months = months_country_mean.index


for year in years:
    mon_obj = {
              "year":year
              }
    for month in months:
        mon_obj[month] = months_country_mean.loc[month , year]        
                
    months_list.append(mon_obj) 
       
        
months_list


In [ ]:
group_temp = temp_df.groupby('Area')
group_temp.mean()